In [2]:
import numpy as np
import pandas as pd
from time import time
from itertools import combinations
import copy 

Juan Manuel Jimenez  
Ian Chona  
Camilo Franco Beltran  


# # Sheet -11-

In [ ]:
Exercise 1 (4 Points)
Consider the following DNA sequence database for  = fA;C; G; Tg:

2.1 Implement functions SPADE(db, minsup) and PrefixSpan(db, minsup) to mine frequent sequences; return a list of tuples (sequences with their support). 

In [55]:
def getP(db,uniques,minsup):#Funcion para obtener el P inicial
    P={}
                
    for unique in uniques: 
        if not(unique in P):
                P[unique]={}
        for tIndex,secuence in enumerate(db):            
            indexes=np.where(secuence==unique)[0]                       
            if indexes.size == 0:
                continue            
            P[unique][tIndex]=indexes
        if len(P[unique])<minsup:
            del P[unique]
    print("ya calcule P")    
    return P

def SPADE(db,minsup): 
    #Obtenemos los elementos unicos de la lista    
    uniques=sorted(set(''.join(db)))   
    #obtenemos P
    db = np.array([ np.array(list(word)) for word in db ])
    P=getP(db,uniques,minsup)
    F=[]
    k=0
    F=SPADErec(P,minsup,F,k)
    return F

def SPADErec(P,minsup,F,k): 
    for ra in P:
        F.append((ra,len(P[ra])))
        Pa={}
        for rb in P:
            newkey=ra+rb[k]
            LposAB=joinIntersection(ra,rb,P,k)
            if not(len(LposAB)<minsup):
                Pa[newkey]=LposAB
        
        if Pa:
            SPADErec(Pa,minsup,F,k+1)
        
    return F       
def joinIntersection(ra,rb,P,k):    
    rab={}    
    for element in P[rb]:               
        if not(element in P[ra]):
            continue
        else:
            Lposa=P[ra][element]
            Lposb=P[rb][element]            
            rab[element]=Lposb[np.where(Lposb>np.min(Lposa))]
            if rab[element].size==0:
                del rab[element]
        
            
    return rab
start=time()    
db=np.loadtxt('capitulo10.dat', dtype=str, delimiter='#', comments=None)
print(SPADE(db,4))
finish=time()  
print("me demore: ",finish-start)

ya calcule P
[('A', 7), ('AC', 6), ('ACT', 4), ('AG', 6), ('AGC', 6), ('AGT', 5), ('AGTC', 4), ('AT', 6), ('ATC', 5), ('C', 7), ('CA', 6), ('CAG', 4), ('CC', 4), ('CG', 6), ('CGC', 4), ('CT', 5), ('CTC', 4), ('G', 7), ('GA', 5), ('GAG', 4), ('GC', 6), ('GCA', 4), ('GCG', 4), ('GG', 4), ('GT', 6), ('GTC', 5), ('T', 7), ('TA', 5), ('TC', 6), ('TCA', 5), ('TCG', 4), ('TG', 5)]
me demore:  0.019526958465576172


In [57]:
def suportElements(elements,Dr,minsup): 
    suports={}
    for element in elements:
        if not(element in suports):
            suports[element]=0
        for secuence in Dr:            
            if element in secuence:
                suports[element]+=1 
        if suports[element]<minsup:
            del suports[element]
    return suports

def PrefixSpan(db,minsup):
    r=''
    F=[] 
    uniques=sorted(set(''.join(db)))
    sigma=suportElements(uniques,db,minsup)
    F=PrefixSpanRec(db,r,minsup,F,sigma)
    
    return F

def PrefixSpanRec(dr,r,minsup,F,sigma):    
    Ds={}    
    concatSupKeys=''.join(sigma.keys())    
    for s in sigma:        
        rs=r+s        
        F.append((rs,sigma[s]))
        Ds=[]
        for si in dr:            
            if not(s in si):
                continue
            subString=si[si.index(s)+1:]            
            for sub in subString:                
                if not(sub in concatSupKeys):                    
                    subString=subString.replace(sub,'')
            if subString=='':
                continue
            Ds.append(subString)
        newuniques=sorted(set(''.join(Ds)))        
        if Ds:
            newSigma=suportElements(newuniques,Ds,minsup)            
            PrefixSpanRec(Ds,rs,minsup,F,newSigma)
            
    return F
start=time() 
db=np.loadtxt('capitulo10.dat', dtype=str, delimiter='#', comments=None).tolist()
print(PrefixSpan(db,4))
finish=time()  
print("me demore: ",finish-start)

[('A', 7), ('AC', 6), ('ACT', 4), ('AG', 6), ('AGC', 6), ('AGT', 5), ('AGTC', 4), ('AT', 6), ('ATC', 5), ('C', 7), ('CA', 6), ('CAG', 4), ('CC', 4), ('CG', 6), ('CGC', 4), ('CT', 5), ('CTC', 4), ('G', 7), ('GA', 5), ('GAG', 4), ('GC', 6), ('GCA', 4), ('GCG', 4), ('GG', 4), ('GT', 6), ('GTC', 5), ('T', 7), ('TA', 5), ('TC', 6), ('TCA', 5), ('TCG', 4), ('TG', 5)]
me demore:  0.004884004592895508


2. Implement a function MineSubstrings(db, minsup), which mines frequent substrings
in the database; return substrings with their support.

In [49]:

def MineSubstrings(db,minsup):
    
    temp=[[""]]    
    for ind,st in enumerate(db):
        s=list("".join(st))
        n=len(s)
        s.append("$")
        for i in range(0,n+1):
            for j in range(0,i+1):
                if ["".join(s[j:i])] in temp:
                    temp[temp.index(["".join(s[j:i])])]=["".join(s[j:i])+s[i]]
                else:
                    temp.append([s[i]])
    
    
    return PrefixSpan(np.array(temp).flatten(),minsup)

db=np.loadtxt('capitulo10.dat', dtype=str, delimiter='#', comments=None).tolist()
print(MineSubstrings(db,3))

[('$', 49), ('A', 29), ('A$', 29), ('AA', 4), ('AA$', 4), ('AC', 16), ('AC$', 16), ('ACA', 4), ('ACA$', 4), ('ACC', 7), ('ACC$', 7), ('ACCA', 3), ('ACCA$', 3), ('ACG', 9), ('ACG$', 9), ('ACGA', 3), ('ACGA$', 3), ('ACGC', 3), ('ACGC$', 3), ('ACGCA', 3), ('ACGCA$', 3), ('ACT', 8), ('ACT$', 8), ('ACTA', 3), ('ACTA$', 3), ('ACTC', 7), ('ACTC$', 7), ('ACTCA', 3), ('ACTCA$', 3), ('ACTG', 3), ('ACTG$', 3), ('AG', 20), ('AG$', 20), ('AGA', 4), ('AGA$', 4), ('AGC', 11), ('AGC$', 11), ('AGCA', 4), ('AGCA$', 4), ('AGCC', 5), ('AGCC$', 5), ('AGCT', 5), ('AGCT$', 5), ('AGCTC', 4), ('AGCTC$', 4), ('AGT', 9), ('AGT$', 9), ('AGTC', 8), ('AGTC$', 8), ('AT', 11), ('AT$', 11), ('ATA', 4), ('ATA$', 4), ('ATC', 10), ('ATC$', 10), ('ATCA', 4), ('ATCA$', 4), ('ATG', 4), ('ATG$', 4), ('C', 35), ('C$', 35), ('CA', 21), ('CA$', 21), ('CAC', 9), ('CAC$', 9), ('CACC', 3), ('CACC$', 3), ('CACG', 5), ('CACG$', 5), ('CACT', 3), ('CACT$', 3), ('CACTC', 3), ('CACTC$', 3), ('CAG', 13), ('CAG$', 13), ('CAGC', 4), ('CAGC

In [75]:
debug=0

def CombinationThree(db,dbOriginal,juegos,numeroJuegos,Puntaje,k):
    if debug==1:
        print("db=",db,"bnoriginal=",dbOriginal,"k=",k+1)
    for eventA in db: 
        
        if db[eventA]>Puntaje:
            continue
        newDb={}
        for eventB in db:
            eventAB=eventA+eventB[k]            
            joinAB=db[eventA]+dbOriginal[eventB[k]]
            if joinAB<=Puntaje:
                newDb[eventAB]=joinAB
            if k+2==numeroJuegos and joinAB==Puntaje:
                newResults={}
                for neutralEvent in dbOriginal:
                     newResults[neutralEvent]=eventAB.count(neutralEvent)                
                if not(newResults in juegos):
                    print ('-'.join('{}'.format(v) for k,v in newResults.items()))                       
                    #print(list(newResults.values()))
                    print("------------------------")
                    juegos.append(newResults)
        if k+2<=numeroJuegos:            
            CombinationThree(newDb,dbOriginal,juegos,numeroJuegos,Puntaje,k+1)           
            
          
    return juegos



def Campeonato(db,numeroJuegos,Puntaje):
    if db['G']*numeroJuegos < Puntaje:
        return 'No es posible tener ese puntaje con tan pocos juegos'
    juegos=[]
    k=0    
    posibilidades=CombinationThree(db,db,juegos,numeroJuegos,Puntaje,k)
    
    return posibilidades
db={'G':3,'E':1,'P':0,'D':2}
#Prueba 1 ,10 juegos, 1 punto
print("RESULTADOS:::",Campeonato(db,20,6))
print("----------------")  
#Prueba 2 , 5 juegos 4 puntos
print("RESULTADOS:::",Campeonato(db,5,4))


2-0-18-0
------------------------
1-3-16-0
------------------------
1-1-17-1
------------------------
0-6-14-0
------------------------
0-4-15-1
------------------------
0-2-16-2
------------------------
0-0-17-3
------------------------
RESULTADOS::: [{'G': 2, 'E': 0, 'P': 18, 'D': 0}, {'G': 1, 'E': 3, 'P': 16, 'D': 0}, {'G': 1, 'E': 1, 'P': 17, 'D': 1}, {'G': 0, 'E': 6, 'P': 14, 'D': 0}, {'G': 0, 'E': 4, 'P': 15, 'D': 1}, {'G': 0, 'E': 2, 'P': 16, 'D': 2}, {'G': 0, 'E': 0, 'P': 17, 'D': 3}]
----------------
1-1-3-0
------------------------
0-4-1-0
------------------------
0-2-2-1
------------------------
0-0-3-2
------------------------
RESULTADOS::: [{'G': 1, 'E': 1, 'P': 3, 'D': 0}, {'G': 0, 'E': 4, 'P': 1, 'D': 0}, {'G': 0, 'E': 2, 'P': 2, 'D': 1}, {'G': 0, 'E': 0, 'P': 3, 'D': 2}]


In [56]:
{'G': 1, 'E': 1, 'P': 3} in [{'G': 11, 'E': 1, 'P': 3},{'G': 11, 'E': 21, 'P': 13}]

False

In [12]:
a=np.array([0,0])
b=np.array([1,2564654])
c=np.array([0,16574123123])
d=np.array([1123123,0])

print(np.any(a==0))
print(np.any(b==0))
print(np.any(c==0))
print(np.any(d==0))


True
False
True
True


In [16]:
#Con listas
a=[0,0]
b=[1,0]
c=[0,1]
d=[12,11]

print(a.count(0)>0)
print(b.count(0)>0)
print(c.count(0)>0)
print(d.count(0)>0)


True
True
True
False


In [123]:
def threeNplusOne(a):
    iterations=1
    if a!=1:
        while a!=1:                        
            if a%2!=0:
                a=3*a+1
            else:
                a=a/2
            iterations+=1        
    return iterations

def maxTimeBetween(a,b):
    maximun=0
    for i in range(a,b):        
        val=threeNplusOne(i)
        if val>maximun:
            maximun=val
    result=str(a)+" "+str(b)+" "+str(maximun)
    return result

maxTimeBetween(201,210)

'201 210 89'